In [1]:
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## 현재까지 구축했던 데이터 불러오기

In [2]:
accident_count_filter_23 <- read.csv('accident_count_filter_23.csv')
dim(accident_count_filter_23)
accident_count_filter_23 %>% head(2)

[1] 5556   34

,X,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,⋯,안전지대수,중앙분리대수,정차금지지대수,도로속도표시수,교통안전표지수,노드개수,횡단보도수,건물면적,자동차대수,총거주인구수
,<int>,<fct>,<int>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
1,0,다바931203,2,"MULTIPOLYGON (((127.4230710131166 36.38013455218083, 127.4230701251944 36.38103608833949, 127.4241850500282 36.38103680113275, 127.424185925082 36.38013526495075, 127.4230710131166 36.38013455218083)))",2,2,127.4236,36.38059,1,3,⋯,1,0,0,2,5,0,1,1291.19,409,0
2,1,다바861174,0,"MULTIPOLYGON (((127.3450791441312 36.35391426501025, 127.3450773577309 36.35481580264442, 127.3461919054753 36.3548172424657, 127.346193679024 36.35391570478436, 127.3450791441312 36.35391426501025)))",0,0,127.3456,36.35437,0,0,⋯,0,0,0,1,0,0,0,1159.69,16,24


## 23. 교통사고내역 데이터를 활용하여 사고유형과 연령대를 현재까지 구축한 데이터와 병합

사고유형(차대차, 차대사람)과 연령대 변수(20대 미만, 20대, 30대, 40대, 50대, 60대 이상)를 모두 고려하기 위하여<br/>
각각의 데이터를 병합하여 **총 12개의 새로운 그룹을 만들고, 전체 데이터와 병합**하도록 한다.

In [3]:
accident_list <- read.csv('1.대전광역시_교통사고내역(2017~2019).csv')
accident_list_sep <- accident_list %>% separate('사고유형', sep='-', into=c('사고유형메인', '사고유형디테일')) # 사고유형 변수를 종류와 디테일로 나누기
dim(accident_list_sep)
accident_list_sep %>% head(2)

Warning message:
“Expected 2 pieces. Additional pieces discarded in 62 rows [618, 651, 1355, 1391, 1492, 1667, 1889, 2035, 2054, 2377, 3088, 3205, 3360, 3448, 3878, 4303, 4405, 5203, 6775, 7388, ...].”


[1] 23652    17

,사고일,시군구,사고유형메인,사고유형디테일,법규위반,사고내용,사망자수,중상자수,경상자수,부상신고자수,가해운전자.차종,가해운전자.연령대,가해운전자.성별,피해운전자.차종,피해운전자.연령대,피해운전자.성별,gid
,<fct>,<fct>,<chr>,<chr>,<fct>,<fct>,<int>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,2017-01-01,대전광역시 서구 용문동,차대차,측면충돌,안전운전불이행,중상사고,0,1,1,0,승용,50대,남,승용,60대,남,다바905151
2,2017-01-01,대전광역시 서구 탄방동,차대차,추돌,안전운전불이행,경상사고,0,0,1,0,승용,50대,남,승용,30대,남,다바905166


In [4]:
accident_list_sep %>%
    group_by(사고유형메인, 피해운전자.연령대, gid) %>% # 사고유형과 연령대 변수에 대하여 격자별 개수 구하기
        summarise(n=n()) %>% ungroup() -> acci_list
dim(acci_list)
acci_list %>% head(2)

`summarise()` regrouping output by '사고유형메인', '피해운전자.연령대' (override with `.groups` argument)



[1] 15767     4

사고유형메인,피해운전자.연령대,gid,n
<chr>,<fct>,<fct>,<int>
차대사람,10대,다바823162,1
차대사람,10대,다바827180,1


In [5]:
acci_list <- acci_list %>% filter(사고유형메인 %in% c('차대사람 ', '차대차 ')) # 차대사람과 차대차에 해당하는 값만 추리기(차량단독 제외)
dim(acci_list)
acci_list %>% head(2)

[1] 15145     4

사고유형메인,피해운전자.연령대,gid,n
<chr>,<fct>,<fct>,<int>
차대사람,10대,다바823162,1
차대사람,10대,다바827180,1


In [6]:
# 피해자의 연령대는 미분류된 것을 제외하고, 20대 미만부터 60대 이상까지의 범주로 재정의
acci_list <- acci_list %>% filter(피해운전자.연령대 != '미분류') 
acci_list$피해운전자.연령대 <- as.character(acci_list$피해운전자.연령대)
acci_list$피해운전자.연령대 <- ifelse(acci_list$피해운전자.연령대 %in% c('60대', '70대', '80대', '90대'), '60대 이상', acci_list$피해운전자.연령대)
acci_list$피해운전자.연령대 <- ifelse(acci_list$피해운전자.연령대 %in% c('10대', '10대미만'), '20대 미만', acci_list$피해운전자.연령대)
dim(acci_list)
acci_list %>% head(2)

[1] 15110     4

사고유형메인,피해운전자.연령대,gid,n
<chr>,<chr>,<fct>,<int>
차대사람,20대 미만,다바823162,1
차대사람,20대 미만,다바827180,1


In [7]:
acci_list <- acci_list %>% unite(사고유형, 사고유형메인, 피해운전자.연령대, sep="-") # 사고유형과 연령대를 묶어주기
dim(acci_list)
acci_list %>% head(3)

[1] 15110     3

사고유형,gid,n
<chr>,<fct>,<int>
차대사람 -20대 미만,다바823162,1
차대사람 -20대 미만,다바827180,1
차대사람 -20대 미만,다바828179,1


In [8]:
acci_list %>%
    group_by(gid, 사고유형) %>% # 격자와 사고유형에 따라서 개수를 구하기
        summarise(n=sum(n)) %>%
            ungroup() -> acci_group
dim(acci_group)
acci_group %>% head(3)

`summarise()` regrouping output by 'gid' (override with `.groups` argument)



[1] 14626     3

gid,사고유형,n
<fct>,<chr>,<int>
다바780093,차대차 -50대,1
다바781090,차대차 -50대,1
다바781091,차대차 -40대,1


In [9]:
acci_spread <- acci_group %>% spread(사고유형, n, fill=0) # 위 데이터의 행과 열을 바꿔주기
dim(acci_spread)
acci_spread %>% head(2)

[1] 5915   13

gid,차대사람 -20대,차대사람 -20대 미만,차대사람 -30대,차대사람 -40대,차대사람 -50대,차대사람 -60대 이상,차대차 -20대,차대차 -20대 미만,차대차 -30대,차대차 -40대,차대차 -50대,차대차 -60대 이상
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
다바780093,0,0,0,0,0,0,0,0,0,0,1,0
다바781090,0,0,0,0,0,0,0,0,0,0,1,0


In [10]:
accident_count_filter_23 <- accident_count_filter_23 %>% left_join(acci_spread, by='gid') # 기존 데이터와 병합

for(i in 34:45){
    accident_count_filter_23[,names(accident_count_filter_23)[i] ] <- ifelse(is.na(accident_count_filter_23[,names(accident_count_filter_23)[i] ]), 0, accident_count_filter_23[,names(accident_count_filter_23)[i] ])
}

dim(accident_count_filter_23)
accident_count_filter_23 %>% head(2)

[1] 5556   46

,X,gid,acci_cnt,geometry,사고건수,사상자수,x,y,신호등_보행자수,신호등_차량등수,⋯,차대사람 -30대,차대사람 -40대,차대사람 -50대,차대사람 -60대 이상,차대차 -20대,차대차 -20대 미만,차대차 -30대,차대차 -40대,차대차 -50대,차대차 -60대 이상
,<int>,<fct>,<int>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,다바931203,2,"MULTIPOLYGON (((127.4230710131166 36.38013455218083, 127.4230701251944 36.38103608833949, 127.4241850500282 36.38103680113275, 127.424185925082 36.38013526495075, 127.4230710131166 36.38013455218083)))",2,2,127.4236,36.38059,1,3,⋯,0,0,0,1,0,0,0,1,0,0
2,1,다바861174,0,"MULTIPOLYGON (((127.3450791441312 36.35391426501025, 127.3450773577309 36.35481580264442, 127.3461919054753 36.3548172424657, 127.346193679024 36.35391570478436, 127.3450791441312 36.35391426501025)))",0,0,127.3456,36.35437,0,0,⋯,0,0,0,0,0,0,0,0,0,NA


In [11]:
write.csv(accident_count_filter_23, 'accident_count_filter_24.csv')
# 신호등(보행등), 신호등(차량등), cctv수, 혼잡빈도, 혼잡시간, 교통추정량, 날씨, 안전지대수, 중앙분리대수, 정차금지지대수
# 도로속도표시수, 교통안전표지수, 노드개수, 횡단보도수, 건물면적, 자동차대수, 총거주인구수, 연령 및 사고유형에 따라 나눈 데이터까지 merge한 상태.